In [1]:

from ultralytics import YOLO
import cv2
import numpy as np
import math
from collections import defaultdict

In [2]:
# Load the YOLOv8 model
model = YOLO('yolov8n-pose.pt')
# 新建颜色数组
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255), (255, 0, 255), (128, 0, 0),
          (0, 128, 0), (0, 0, 128), (128, 128, 0), (0, 128,
                                                    128), (128, 0, 128), (128, 128, 128), (64, 0, 0),
          (0, 64, 0), (0, 0, 64), (64, 64, 0), (0, 64, 64), (64, 0, 64), (64, 64, 64)]

# Open the video file
video_path = "E:\BaiduSyncdisk\PY\DataSet\视频\9月14日.mp4"

In [3]:
cap = cv2.VideoCapture(video_path)

In [88]:
success, frame = cap.read()
 # 对frame缩小比例
annotated_frame = cv2.resize(frame, (640, 480))

# Run YOLOv8 tracking on the frame, persisting tracks between frames
results = model.track(annotated_frame, persist=True)

# Get the boxes and track IDs
boxes = results[0].boxes.xywh.cpu().tolist()
cls_list = results[0].boxes.cls.cpu().tolist()
track_ids = results[0].boxes.id.int().cpu().tolist()
keypoints = results[0].keypoints.xy.cpu()


0: 480x640 1 person, 218.0ms
Speed: 10.0ms preprocess, 218.0ms inference, 9.0ms postprocess per image at shape (1, 3, 480, 640)


In [84]:
results[0].boxes.cls.cpu().tolist()

[0.0]

In [89]:
for i in range(keypoints.shape[0]):
    points=keypoints[i].tolist()
    track_id=track_ids[i]
    box=boxes[i]
    cls =cls_list[i]
    name = results[0].names[cls]
    p1 = points[1]
    p2 = points[2]
    p3 = points[3]
    p4 = points[4]
    a = (int((p1[0] + p2[0]) / 2), int((p1[1] + p2[1]) / 2))
    b = (int((p3[0] + p4[0]) / 2), int((p3[1] + p4[1]) / 2))
  
    # 计算圆的半径
    radius = 1.5 * max(math.sqrt((points[0][0] - p3[0]) ** 2 + (points[0][1] - p3[1]) ** 2),
                        math.sqrt((points[0][0] - p4[0]) ** 2 + (points[0][1] - p4[1]) ** 2))
    
    # 计算线段的终点坐标
    angle = math.atan2(a[1] - b[1], a[0] - b[0])
    endpointX =(int)( b[0] + radius * math.cos(angle))
    endpointY = (int)(b[1] + radius * math.sin(angle))
    endpoint = (endpointX, endpointY)
    
    cv2.line(annotated_frame, (b[0],b[1]),(endpoint[0],endpoint[1]),colors[track_id % len(colors)], 2)
